In [ ]:
# import pickle
# df_llc_sparse = dat.df_llc_sparse
# df_loss_sparse = dat.df_loss_sparse

# data_path = "data"
# print("warning - patching in large model 1024 ctx as if full ctx.")

# # Sparse losses - contains results for 1B but doesn't contain std
# with open(f'{data_path}/variable_context_pile_subsets.pkl',
#           'rb') as file:
#     df_big = pickle.load(file)

# keep = [c for c in df_big.columns if "_1024_" in c]
# rename = ["pythia-" + c.replace("_pile-", "-").replace("_1024_", "_").replace("_pile1m", "-full") for c in keep]
# df_big.set_index('Step', inplace=True)
# df_big = df_big[keep]
# df_big.rename(columns=dict(zip(keep, rename)), inplace=True)

# # Now split into loss and LLC columns
# loss_cols = [c for c in df_big if c.endswith("_loss")]
# base = [c[:-5] for c in loss_cols]
# llc_cols = [c + "_llc" for c in base]

# patch_loss = df_big[loss_cols].rename(columns=dict(zip(loss_cols, base)))
# patch_llc = df_big[llc_cols].rename(columns=dict(zip(llc_cols, base)))

# df_llc_sparse = df_llc_sparse.join(patch_llc)
# df_loss_sparse = df_loss_sparse.join(patch_loss)

# df_llc_sparse

make a plot of (time, loss), (time, llc), (llc, loss) for dm_maths with all models we now have data for, each corresponding to a line coloured with Viridian? (14m to 6.9b I think)

In [ ]:
%load_ext autoreload
%autoreload 2
import data_utils as dat
import vis
import plotly.graph_objects as go  # temporarily don't drop
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
dat.reload_data("data")
llc_desc = r"$\text{Estimated }\,\hat{\lambda}$"
loss_desc = r"$\text{Loss }L$"

models = ['14m', '31m', '70m', '160m', '410m', '1b', '1.4b', '2.8b', '6.9b']

task = "dm_mathematics"
colors = vis.assign_cols(models, cmap="viridis", seed=42, shuffle=False)
step = dat.df_loss_sparse.index.values

fig = go.Figure()

for model in models:
    color = colors[model]
    name = f"pythia-{model}-{task}"
    loss = dat.df_loss_sparse[name].values
    llc = dat.df_llc_sparse[name].values
    
    fig.add_trace(
        go.Scatter(
            x=step,
            y=llc,
            name="Pythia-"+model,
            mode="markers+lines",
            marker=dict(color=color),
    ))
fig.update_xaxes(type="log")
fig.update_xaxes(title_text="step")
fig.update_yaxes(title_text=llc_desc)
fig.update_layout(
    width=1000,
    height=600,
    title="DM_Mathematics: LLC vs time"
)
fname = "plots/dm_llc_vs_time"
fig.write_html(fname + ".html")
fig.write_image(fname+ ".png", scale=2)
fig.show()

In [ ]:
tasks  = [c[c.find("-")+1:] for c in dat.df_llc_sparse.columns]
tasks = [c[c.find("-")+1:] for c in tasks]
tasks = list(set(tasks))
tasks

# pile_subsets = [
#  'pile1m',
#  'pile-github',
#  'pile-pile-cc',
#  'pile-pubmed_abstracts',
#  'pile-uspto_backgrounds',
#  'pile-pubmed_central',
#  'pile-stackexchange',
#  'pile-wikipedia_en',
#  'pile-freelaw',
#  'pile-arxiv',
#  'pile-dm_mathematics',
#  'pile-nih_exporter',
#  'pile-enron_emails',
#  'pile-hackernews',
# ]


In [ ]:
fig = go.Figure()

for model in models:
    color = colors[model]
    name = f"pythia-{model}-{task}"
    loss = dat.df_loss_sparse[name].values
    llc = dat.df_llc_sparse[name].values
    
    fig.add_trace(
        go.Scatter(
            x=step,
            y=loss,
            name="Pythia-"+model,
            mode="markers+lines",
            marker=dict(color=color),
    ))
fig.update_xaxes(type="log")
fig.update_xaxes(title_text="step")
fig.update_yaxes(title_text=loss_desc)
fig.update_layout(
    width=1000,
    height=600,
    title="DM_Mathematics: loss vs time"
)
fname = "plots/dm_loss_vs_time"
fig.write_html(fname + ".html")
fig.write_image(fname+ ".png", scale=2)
fig.show()

In [ ]:
fig = go.Figure()
start = np.argmax(step>100)
step = dat.df_loss_sparse.index.values


for model in models:
    color = colors[model]
    name = f"pythia-{model}-{task}"
    loss = dat.df_loss_sparse[name].values
    llc = dat.df_llc_sparse[name].values
    name="Pythia-"+model

    vis.plot_data(fig, llc[start:], loss[start:], step[start:], color=color, name=name, mode="markers+lines")

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

#fig.update_xaxes(type="log")
fig.update_xaxes(title_text=llc_desc)
fig.update_yaxes(title_text=loss_desc)
fig.update_layout(
    width=800,
    height=800,
    title="DM_Mathematics: loss vs llc"
)
fname = "plots/dm_loss_vs_llc"
fig.write_html(fname + ".html", include_mathjax='cdn')
fig.write_image(fname+ ".png", scale=2)


fig.show()

In [ ]:
# DO them all as subplots then...

base = "DM_Mathematics: "
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(base+'Loss vs Step', base+'LLC vs Step', base+'Loss vs LLC'),
    vertical_spacing=0.05,
)
A, B, C = [dict(row=1+i, col=1) for i in range(3)]

for model in models:
    color = colors[model]
    name = f"pythia-{model}-{task}"
    desc = f"pythia-{model}(1024)"
    loss = dat.df_loss_sparse[name].values
    llc = dat.df_llc_sparse[name].values
    name="Pythia-"+model
    
    vis.plot_data(fig, step, loss, step, color=color, name=desc, showlegend=False, mode="markers+lines", subplot=A)
    vis.plot_data(fig, step, llc, step, color=color, name=desc, showlegend=False, mode="markers+lines", subplot=B)
    vis.plot_data(fig, llc, loss, step, color=color, name=desc, mode="markers+lines", subplot=C)
    
# A: loss vs step
fig.update_xaxes(type="log", **A)
fig.update_xaxes(title_text="Step", **A)
fig.update_yaxes(title_text=loss_desc, **A)

# B: LLC vs step
fig.update_xaxes(type="log", **B)
fig.update_xaxes(title_text="Step", **B)
fig.update_yaxes(title_text=llc_desc, **B)

# C: LLC vs time
fig.update_xaxes(type="log", **C)
fig.update_yaxes(type="log", **C)
fig.update_xaxes(title_text=llc_desc, **C)
fig.update_yaxes(title_text=loss_desc, **C)
fig.update_layout(
    width=1000,
    height=3*800,
    title="DM_Mathematics - Model checkpoints @ n_ctx=1024"
)
fname = "plots/dm_all"
fig.write_html(fname + ".html", include_mathjax='cdn')
fig.write_image(fname+ ".png", scale=2)


fig.show()

In [ ]:
# DO ALL THE TASKS as dropdowns...

titles = [
    tasks[0] + ': Loss vs Step',
    tasks[0] + ': LLC vs Step',
    tasks[0] + ': Loss vs LLC',
]
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=titles,
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
)
A, B, C = [dict(row=1+i, col=1) for i in range(3)]

task_ind = []  # keep track of which task corresponds to every dataset

for j, task in enumerate(tasks):
    # Address the 3 subplots corresponding to this task
    for model in models:
        # Put all models on the subplots
        color = colors[model]
        name = f"pythia-{model}-{task}"
        desc = f"pythia-{model}(1024)"
        loss = dat.df_loss_sparse[name].values
        llc = dat.df_llc_sparse[name].values
        name="Pythia-"+model
        v = (j==0)
        vis.plot_data(fig, step, loss, step, color=color, name=desc, showlegend=False, mode="markers+lines", subplot=A, visible=v)
        vis.plot_data(fig, step, llc, step, color=color, name=desc, showlegend=False, mode="markers+lines", subplot=B, visible=v)
        vis.plot_data(fig, llc, loss, step, color=color, name=desc, mode="markers+lines", subplot=C, visible=v)
        task_ind.extend([j, j, j])  # 3 plots on task j

# A: loss vs step
fig.update_xaxes(type="log", **A)
fig.update_xaxes(title_text="Step", **A)
fig.update_yaxes(title_text=loss_desc, **A)

# B: LLC vs step
fig.update_xaxes(type="log", **B)
fig.update_xaxes(title_text="Step", **B)
fig.update_yaxes(title_text=llc_desc, **B)

# C: LLC vs time
fig.update_xaxes(type="log", **C)
fig.update_yaxes(type="log", **C)
fig.update_xaxes(title_text=llc_desc, **C)
fig.update_yaxes(title_text=loss_desc, **C)

fig.update_layout(
    width=1000,
    height=3*600,
    title="Model checkpoints @ n_ctx=1024"
)

# Make the dropdown settings
buttons = []
for j, task in enumerate(tasks):
    buttons.append(dict(
        label=task,
        method="update",
        args=[
            {"visible": [t == j for t in task_ind]},
            {
                "annotations[0].text": task + ': Loss vs Step',
                "annotations[1].text": task + ': LLC vs Step',
                "annotations[2].text": task + ': Loss vs LLC',
            }
        ]
    ))

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.45,
            y=1.045,
            xanchor="left",
            yanchor="top",
            type="dropdown"
        ),
    ]
)

fname = "plots/dropdown_all"
fig.write_html(fname + ".html", include_mathjax='cdn')
fig.show()

In [ ]:
# DO ALL THE TASKS as subplots too...
titles = []
for task in tasks:
    titles.extend((task+': Loss vs Step', task+': LLC vs Step', task+': Loss vs LLC'))

n_rows = len(tasks)
n_cols = 3
fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=titles,
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
)
# Refer to these plots
A, B, C = [dict(row=1+r, col=1) for r in range(3)]


for j, task in enumerate(tasks):
    # Address the 3 subplots corresponding to this task
    

    for model in models:
        # Put all models on the subplots
        color = colors[model]
        name = f"pythia-{model}-{task}"
        desc = f"pythia-{model}(1024)"
        loss = dat.df_loss_sparse[name].values
        llc = dat.df_llc_sparse[name].values
        name="Pythia-"+model
        
        vis.plot_data(fig, step, loss, step, color=color, name=desc, showlegend=False, mode="markers+lines", subplot=A)
        vis.plot_data(fig, step, llc, step, color=color, name=desc, showlegend=False, mode="markers+lines", subplot=B)
        vis.plot_data(fig, llc, loss, step, color=color, name=desc, mode="markers+lines", subplot=C)
        
    # A: loss vs step
    fig.update_xaxes(type="log", **A)
    fig.update_xaxes(title_text="Step", **A)
    fig.update_yaxes(title_text=loss_desc, **A)
    
    # B: LLC vs step
    fig.update_xaxes(type="log", **B)
    fig.update_xaxes(title_text="Step", **B)
    fig.update_yaxes(title_text=llc_desc, **B)
    
    # C: LLC vs time
    fig.update_xaxes(type="log", **C)
    fig.update_yaxes(type="log", **C)
    fig.update_xaxes(title_text=llc_desc, **C)
    fig.update_yaxes(title_text=loss_desc, **C)

fig.update_layout(
    width=n_cols*600,
    height=n_rows*600,
    title="Model checkpoints @ n_ctx=1024"
)
fname = "plots/all_all"
fig.write_html(fname + ".html", include_mathjax='cdn')
fig.write_image(fname+ ".png", scale=2)


fig.show()

# 1. When are the "mode switches"
# 2. What are the evals doing at this stage? What about the latent cababilities ala Burnell?
(Maybe we pick those methods within the eleuther evals as we already have them)
# 3. What about the losses on our "easy" dataset?


In [ ]:
#1. Eleuther evals (easy)
pythia v0 or v1
deduped or not?

in pythia-v0 we get 70m, 160m, 

Just do factors at the end point